---
title: "Structural Estimation for MDP"
subtitle: "Parameter Recovery from Simulated Panel Data"
format:
  html:
    code-fold: false
    toc: true
    toc-depth: 3
---

## Estimation Problem

### Objective

Given panel data $\{(s_{it}, a_{it})\}_{i=1,\ldots,N; t=0,\ldots,T-1}$ of states and actions, we seek to recover the structural parameters $\theta = (\beta, \gamma, \delta)$ that generated the data.

### Model Specification

The data-generating process follows the dynamic discrete choice model:

- **State transition**: $s_{t+1} = (1 - \gamma) s_t + a_t$ (deterministic)
- **Flow reward**: $u(s, a) = \beta \log(1 + s) - a$
- **Choice-specific value function**:
$$
v(s, a; \theta) = u(s, a) + \delta \bar{V}((1-\gamma)s + a; \theta)
$$
- **Integrated value function** (log-sum-exp):
$$
\bar{V}(s; \theta) = \log\left( \exp(v(s, 0; \theta)) + \exp(v(s, 1; \theta)) \right) + \gamma_E
$$
where $\gamma_E \approx 0.5772$ is Euler's constant.

### Choice Probability

Under Type-I Extreme Value shocks, the probability of choosing action $a = 1$ given state $s$ is:
$$
P(a = 1 | s; \theta) = \frac{\exp(v(s, 1; \theta))}{\exp(v(s, 0; \theta)) + \exp(v(s, 1; \theta))} = \frac{1}{1 + \exp(v(s, 0; \theta) - v(s, 1; \theta))}
$$

This is the **logit formula** with choice probability determined by the value difference.

## Maximum Likelihood Estimation

### Likelihood Function

The log-likelihood of observing the panel data is:
$$
\mathcal{L}(\theta) = \sum_{i=1}^{N} \sum_{t=0}^{T-1} \log P(a_{it} | s_{it}; \theta)
$$

Expanding the choice probability:
$$
\mathcal{L}(\theta) = \sum_{i=1}^{N} \sum_{t=0}^{T-1} \left[ a_{it} \cdot v(s_{it}, 1; \theta) + (1 - a_{it}) \cdot v(s_{it}, 0; \theta) - \log\left( \exp(v(s_{it}, 0; \theta)) + \exp(v(s_{it}, 1; \theta)) \right) \right]
$$

Or equivalently:
$$
\mathcal{L}(\theta) = \sum_{i=1}^{N} \sum_{t=0}^{T-1} \left[ a_{it} \cdot \Delta v(s_{it}; \theta) - \log\left( 1 + \exp(\Delta v(s_{it}; \theta)) \right) \right]
$$
where $\Delta v(s; \theta) = v(s, 1; \theta) - v(s, 0; \theta)$ is the value difference.

### MLE Estimator

The maximum likelihood estimator is:
$$
\hat{\theta}_{MLE} = \arg\max_{\theta} \mathcal{L}(\theta)
$$

### Computational Challenge

The key challenge is that evaluating $\mathcal{L}(\theta)$ requires computing the value functions $v(s, a; \theta)$, which themselves depend on $\theta$ through the Bellman equation. This creates a **nested optimization problem**.

## Our Approach: NFXP with Existing Solver

### Key Insight

We have already implemented a neural network-based value function solver in `mdp_solver`. This solver can be directly invoked for any candidate parameters $\theta = (\beta, \gamma, \delta)$. The estimation problem becomes straightforward:

- **Inner loop**: Call our existing `solve_value_function(beta, gamma, delta, ...)` to obtain converged value networks
- **Outer loop**: Use the value networks to compute choice probabilities and optimize the likelihood

### Available Infrastructure

From `src/mdp_solver`, we have:

| Function | Purpose |
|----------|---------|
| `solve_value_function(beta, gamma, delta, ...)` | Solves Bellman iteration for given parameters |
| `compute_choice_probability(v0_net, v1_net, s)` | Computes $P(a=1|s)$ from value networks |
| `build_monotonic_network(hidden_sizes)` | Creates network architecture |
| `evaluate_network(net, s)` | Evaluates value function at states |

### Algorithm: Two-Step NFXP-NN Estimator

Due to weak identification between parameters when estimating all three simultaneously, we adopt a **two-step approach**:

1. **Step 1**: Estimate $\gamma$ directly from transition data (OLS, no optimization needed)
2. **Step 2**: Estimate $(\beta, \delta)$ via grid search with $\gamma$ fixed

This exploits the fact that $\gamma$ enters only the deterministic transition equation, while $(\beta, \delta)$ enter the value function through the Bellman equation.

#### Pseudocode: Step 1 - Estimate γ from Transitions (OLS)

```
FUNCTION ESTIMATE_GAMMA_OLS(data):
    """
    Estimate γ directly from observed state transitions using OLS.
    
    The transition equation is deterministic:
        s_{t+1} = (1 - γ) * s_t + a_t
    
    Rearranging:
        (s_{t+1} - a_t) = (1 - γ) * s_t
    
    This is a regression through the origin:
        Y = (1 - γ) * X
    where Y = s_{t+1} - a_t, X = s_t
    
    Inputs:
        data: PanelData object containing:
            - states: np.ndarray of shape (n_agents, n_periods)
            - actions: np.ndarray of shape (n_agents, n_periods)
    
    Outputs:
        gamma_hat: float, estimated decay rate
        std_error: float, standard error of estimate
    """
    
    # Extract current states, next states, and actions
    s_current = data.states[:, :-1].flatten()    # s_t (exclude last period)
    s_next = data.states[:, 1:].flatten()        # s_{t+1} (exclude first period)
    a_current = data.actions[:, :-1].flatten()   # a_t (exclude last period)
    
    # Construct regression variables
    Y = s_next - a_current    # Dependent variable
    X = s_current             # Independent variable (regressor)
    
    # Filter out observations where s_t = 0 (division issue)
    valid = X > 1e-6
    Y = Y[valid]
    X = X[valid]
    
    # OLS through origin: β = (X'X)^{-1} X'Y = Σ(X*Y) / Σ(X²)
    coef = np.sum(X * Y) / np.sum(X ** 2)  # This estimates (1 - γ)
    gamma_hat = 1 - coef
    
    # Compute standard error
    residuals = Y - coef * X
    n = len(X)
    mse = np.sum(residuals ** 2) / (n - 1)  # Mean squared error
    var_coef = mse / np.sum(X ** 2)         # Variance of coefficient
    std_error = np.sqrt(var_coef)            # SE for (1-γ), same as SE for γ
    
    RETURN gamma_hat, std_error
```

#### Pseudocode: Step 2 - Grid Search for (β, δ) with γ Fixed

```
FUNCTION ESTIMATE_BETA_DELTA_GRID(data, gamma_fixed, solver_params, bounds, n_points):
    """
    Estimate (β, δ) via 2D grid search with γ fixed.
    
    Inputs:
        data: PanelData object
        gamma_fixed: float, pre-estimated γ from Step 1
        solver_params: dict with solver hyperparameters
        bounds: dict with {'beta': (lo, hi), 'delta': (lo, hi)}
        n_points: int, grid points per dimension
    
    Outputs:
        beta_hat: float, estimated reward coefficient
        delta_hat: float, estimated discount factor
        log_likelihood: float, maximized log-likelihood
    """
    
    # Create 2D grid
    beta_grid = np.linspace(bounds['beta'][0], bounds['beta'][1], n_points)
    delta_grid = np.linspace(bounds['delta'][0], bounds['delta'][1], n_points)
    
    best_ll = -np.inf
    best_beta = None
    best_delta = None
    
    # Search over grid
    FOR beta IN beta_grid:
        FOR delta IN delta_grid:
            theta = (beta, gamma_fixed, delta)
            ll = COMPUTE_LOG_LIKELIHOOD(theta, data, solver_params)
            
            IF ll > best_ll:
                best_ll = ll
                best_beta = beta
                best_delta = delta
    
    RETURN best_beta, best_delta, best_ll
```

#### Pseudocode: Main Two-Step Estimation Routine

```
FUNCTION ESTIMATE_TWO_STEP(data, solver_params, bounds, n_points):
    """
    Two-step structural estimation.
    
    Step 1: Estimate γ from transition data (OLS)
    Step 2: Estimate (β, δ) via grid search with γ fixed
    
    Inputs:
        data: PanelData object
        solver_params: dict with solver hyperparameters
        bounds: dict with parameter bounds
        n_points: int, grid points per dimension for (β, δ)
    
    Outputs:
        theta_hat: tuple (beta_hat, gamma_hat, delta_hat)
        results: dict with estimation details
    """
    
    # Step 1: Estimate γ from transitions
    gamma_hat, gamma_se = ESTIMATE_GAMMA_OLS(data)
    PRINT "Step 1: γ_hat =", gamma_hat, "± ", gamma_se
    
    # Step 2: Estimate (β, δ) with γ fixed
    beta_hat, delta_hat, log_lik = ESTIMATE_BETA_DELTA_GRID(
        data, gamma_hat, solver_params, bounds, n_points
    )
    PRINT "Step 2: β_hat =", beta_hat, ", δ_hat =", delta_hat
    
    theta_hat = (beta_hat, gamma_hat, delta_hat)
    RETURN theta_hat, {'gamma_se': gamma_se, 'log_likelihood': log_lik}
```

---

### Alternative: Joint 3-Parameter Estimation (For Reference)

The following pseudocode shows joint estimation of all three parameters, which suffers from identification issues as demonstrated above.

#### Pseudocode: Joint Estimation (Not Recommended)

```
FUNCTION ESTIMATE_MLE(data, theta_init, solver_params, bounds):
    """
    Estimate structural parameters via Maximum Likelihood.
    
    Inputs:
        data: PanelData object containing:
            - states: np.ndarray of shape (n_agents, n_periods)
            - actions: np.ndarray of shape (n_agents, n_periods)
        theta_init: tuple (beta_init, gamma_init, delta_init)
        solver_params: dict containing:
            - s_min: float, minimum state value
            - s_max: float, maximum state value
            - hidden_sizes: list[int], network architecture
            - learning_rate: float
            - batch_size: int
            - tolerance: float, convergence tolerance for inner loop
            - max_iterations: int, max iterations for inner loop
            - target_update_freq: int
        bounds: list of tuples [(beta_lo, beta_hi), (gamma_lo, gamma_hi), (delta_lo, delta_hi)]
    
    Outputs:
        theta_hat: tuple (beta_hat, gamma_hat, delta_hat)
        result: scipy.optimize.OptimizeResult object
    """
    
    # Define objective function (negative log-likelihood)
    FUNCTION neg_log_likelihood(theta):
        log_lik = COMPUTE_LOG_LIKELIHOOD(theta, data, solver_params)
        RETURN -log_lik
    
    # Run Nelder-Mead optimization
    result = scipy.optimize.minimize(
        fun=neg_log_likelihood,
        x0=theta_init,
        method='Nelder-Mead',
        options={
            'maxiter': 200,
            'xatol': 1e-4,      # Parameter tolerance
            'fatol': 1e-4,      # Function value tolerance
            'disp': True,
            'adaptive': True    # Adapt simplex to parameter scales
        }
    )
    
    theta_hat = result.x
    RETURN theta_hat, result
```

#### Pseudocode: Log-Likelihood Computation (Inner Loop)

```
FUNCTION COMPUTE_LOG_LIKELIHOOD(theta, data, solver_params):
    """
    Compute log-likelihood for candidate parameters.
    This is called once per outer loop iteration.
    
    Inputs:
        theta: tuple (beta, gamma, delta)
        data: PanelData object
        solver_params: dict with solver hyperparameters
    
    Outputs:
        log_lik: float, total log-likelihood
    """
    
    # Unpack parameters
    beta, gamma, delta = theta
    
    # Validate parameter bounds (return -inf for invalid)
    IF beta <= 0 OR gamma <= 0 OR gamma >= 1 OR delta <= 0 OR delta >= 1:
        RETURN -np.inf
    
    # === INNER LOOP: Solve value functions for this theta ===
    v0_net, v1_net, losses, n_iter = solve_value_function(
        beta=beta,
        gamma=gamma,
        delta=delta,
        s_min=solver_params['s_min'],
        s_max=solver_params['s_max'],
        hidden_sizes=solver_params['hidden_sizes'],
        learning_rate=solver_params['learning_rate'],
        batch_size=solver_params['batch_size'],
        tolerance=solver_params['tolerance'],
        max_iterations=solver_params['max_iterations'],
        target_update_freq=solver_params['target_update_freq']
    )
    
    # Check convergence (optional: warn if not converged)
    IF n_iter == solver_params['max_iterations']:
        PRINT warning: "Inner loop did not converge for theta =", theta
    
    # === Compute choice probabilities at all observed states ===
    # Flatten panel data for batch computation
    states_flat = data.states.flatten()  # shape: (N * T,)
    actions_flat = data.actions.flatten()  # shape: (N * T,)
    
    # Convert to tensor
    s_tensor = torch.tensor(states_flat, dtype=torch.float32)
    
    # Compute P(a=1 | s) for all observations
    WITH torch.no_grad():
        p1 = compute_choice_probability(v0_net, v1_net, s_tensor)
        p1 = p1.numpy()
    
    # Clip probabilities for numerical stability
    eps = 1e-10
    p1 = np.clip(p1, eps, 1 - eps)
    
    # === Compute log-likelihood ===
    # L = Σ [a * log(p1) + (1-a) * log(1-p1)]
    log_lik = np.sum(
        actions_flat * np.log(p1) + 
        (1 - actions_flat) * np.log(1 - p1)
    )
    
    RETURN log_lik
```

#### Pseudocode: Standard Error Computation

```
FUNCTION COMPUTE_STANDARD_ERRORS(theta_hat, data, solver_params, eps=1e-4):
    """
    Compute standard errors via numerical Hessian at the MLE.
    
    Inputs:
        theta_hat: tuple (beta_hat, gamma_hat, delta_hat), MLE estimates
        data: PanelData object
        solver_params: dict
        eps: float, step size for finite differences
    
    Outputs:
        std_errors: np.ndarray of shape (3,)
        cov_matrix: np.ndarray of shape (3, 3)
    """
    
    n_params = 3
    theta_hat = np.array(theta_hat)
    
    # Compute numerical Hessian via central differences
    hessian = np.zeros((n_params, n_params))
    
    FOR i IN range(n_params):
        FOR j IN range(n_params):
            # f(θ + ei*eps + ej*eps)
            theta_pp = theta_hat.copy()
            theta_pp[i] += eps
            theta_pp[j] += eps
            f_pp = COMPUTE_LOG_LIKELIHOOD(theta_pp, data, solver_params)
            
            # f(θ + ei*eps - ej*eps)
            theta_pm = theta_hat.copy()
            theta_pm[i] += eps
            theta_pm[j] -= eps
            f_pm = COMPUTE_LOG_LIKELIHOOD(theta_pm, data, solver_params)
            
            # f(θ - ei*eps + ej*eps)
            theta_mp = theta_hat.copy()
            theta_mp[i] -= eps
            theta_mp[j] += eps
            f_mp = COMPUTE_LOG_LIKELIHOOD(theta_mp, data, solver_params)
            
            # f(θ - ei*eps - ej*eps)
            theta_mm = theta_hat.copy()
            theta_mm[i] -= eps
            theta_mm[j] -= eps
            f_mm = COMPUTE_LOG_LIKELIHOOD(theta_mm, data, solver_params)
            
            # Second derivative approximation
            hessian[i, j] = (f_pp - f_pm - f_mp + f_mm) / (4 * eps * eps)
    
    # Covariance matrix = inverse of negative Hessian (information matrix)
    info_matrix = -hessian
    cov_matrix = np.linalg.inv(info_matrix)
    
    # Standard errors = sqrt of diagonal
    std_errors = np.sqrt(np.diag(cov_matrix))
    
    RETURN std_errors, cov_matrix
```

### Why Nelder-Mead

For our 3-parameter problem, Nelder-Mead is the practical choice:

1. **No gradients required**: Computing $\partial \mathcal{L}/\partial \theta$ would require differentiating through thousands of inner loop iterations — complex and numerically unstable.

2. **Robust**: Nelder-Mead handles non-smooth objective functions and doesn't get stuck on saddle points.

3. **Sufficient for low dimensions**: With only 3 parameters, Nelder-Mead converges in reasonable time (~50-200 function evaluations).

4. **Each evaluation is expensive**: Our inner loop (solving the DP) dominates computation time. Nelder-Mead minimizes the number of evaluations.

### Computational Cost

Each outer loop iteration requires:
- 1 call to `solve_value_function` (inner loop): ~1000-10000 iterations of neural network training
- 1 batch evaluation of choice probabilities: O(N × T) forward passes

Total estimation: ~50-200 outer iterations × inner loop cost

## Identification

### Identification Conditions

For the parameters $(\beta, \gamma, \delta)$ to be identified from choice data:

1. **Reward normalization**: One parameter must be normalized. Typically, the coefficient on the action cost is set to 1, so the reward is $u(s,a) = \beta \log(1+s) - a$ (the $-a$ term is normalized to have coefficient -1).

2. **Variation in states**: The data must contain sufficient variation in states $s$ to identify $\beta$ from the state-dependent reward component $\beta \log(1+s)$.

3. **Discount factor**: $\delta$ is identified from the forward-looking behavior. Higher $\delta$ means agents put more weight on future states, affecting current choices.

4. **Transition parameter**: $\gamma$ is directly identified from observed state transitions:
$$
\gamma = 1 - \frac{s_{t+1} - a_t}{s_t} \quad \text{(when } s_t \neq 0 \text{)}
$$

### Exclusion Restrictions

In our model:
- $\gamma$ enters only the transition equation, not the flow reward
- $\beta$ enters only the flow reward, not the transition
- $\delta$ appears only in the discounting of future values

This separation aids identification.

### Special Case: Direct $\gamma$ Estimation

Since state transitions are deterministic, $\gamma$ can be estimated directly from observed transitions without solving the DP problem:
$$
\hat{\gamma} = 1 - \frac{1}{|\mathcal{D}|} \sum_{(s_t, a_t, s_{t+1}) \in \mathcal{D}} \frac{s_{t+1} - a_t}{s_t}
$$

This reduces the estimation problem to finding $(\beta, \delta)$ given known $\gamma$.

## Score and Hessian

### Score Function

The score (gradient of log-likelihood) with respect to $\theta$ is:
$$
S(\theta) = \frac{\partial \mathcal{L}}{\partial \theta} = \sum_{i,t} \left( a_{it} - P(a=1|s_{it}; \theta) \right) \cdot \frac{\partial \Delta v(s_{it}; \theta)}{\partial \theta}
$$

This has the intuitive form: prediction error $\times$ sensitivity of value difference to parameters.

### Information Matrix

The expected (Fisher) information matrix is:
$$
\mathcal{I}(\theta) = -\mathbb{E}\left[ \frac{\partial^2 \mathcal{L}}{\partial \theta \partial \theta'} \right] = \sum_{i,t} P(1-P) \cdot \frac{\partial \Delta v}{\partial \theta} \cdot \frac{\partial \Delta v}{\partial \theta'}
$$
where $P = P(a=1|s_{it}; \theta)$.

### Asymptotic Distribution

Under regularity conditions, the MLE is asymptotically normal:
$$
\sqrt{NT}(\hat{\theta} - \theta_0) \xrightarrow{d} \mathcal{N}(0, \mathcal{I}(\theta_0)^{-1})
$$

Standard errors can be computed from the inverse of the estimated information matrix.

### Numerical Standard Errors

For derivative-free optimization, standard errors can be computed via:
1. **Numerical Hessian**: Finite differences around $\hat{\theta}$
2. **Bootstrap**: Resample the panel data and re-estimate
3. **Outer product of gradients (OPG)**: If numerical gradients are available

## Alternative Approaches (For Reference)

### Alternative 1: Hotz-Miller CCP Inversion

Instead of solving the full dynamic problem, Hotz and Miller (1993) showed that value differences can be expressed in terms of observable CCPs:
$$
v(s, 1) - v(s, 0) = \log\left( \frac{P(a=1|s)}{P(a=0|s)} \right)
$$

**Two-Step Estimation**:

1. **First stage**: Estimate CCPs non-parametrically from the data:
$$
\hat{P}(a=1|s) = \frac{\sum_{i,t} \mathbf{1}[a_{it}=1, s_{it} \approx s]}{\sum_{i,t} \mathbf{1}[s_{it} \approx s]}
$$

2. **Second stage**: Use the inverted CCPs to form moment conditions and estimate structural parameters.

**Pros**: Avoids solving the Bellman equation repeatedly
**Cons**: Requires accurate first-stage CCP estimates, which introduces noise — particularly problematic with limited data or sparse state coverage

### Alternative 2: Simulation-Based Estimation (Indirect Inference)

Simulate data from the model at candidate parameters and match moments:
$$
\hat{\theta} = \arg\min_{\theta} \left( m(\text{data}) - m(\text{simulated}(\theta)) \right)' W \left( m(\text{data}) - m(\text{simulated}(\theta)) \right)
$$

**Pros**: Flexible, can match any computable moments
**Cons**: Requires many simulations, choice of moments affects efficiency

### Alternative 3: Bayesian Estimation

Place priors on $\theta$ and compute posterior:
$$
p(\theta | \text{data}) \propto \mathcal{L}(\theta) \cdot p(\theta)
$$

**Pros**: Quantifies parameter uncertainty, regularizes estimation
**Cons**: Computationally intensive (MCMC), requires prior specification

## Summary of Estimation Approaches

| Approach | Pros | Cons |
|----------|------|------|
| **NFXP-NN (Our Approach)** | Reuses existing solver, exact probabilities, no CCP noise | Requires solving DP per evaluation |
| **Hotz-Miller CCP** | Avoids repeated DP | Noisy CCP estimates, binning artifacts |
| **Indirect Inference** | Flexible moment matching | Many simulations needed |
| **Bayesian** | Full uncertainty quantification | MCMC computational cost |

## Implementation Roadmap

### Module Structure

```
src/mdp_estimator/
├── __init__.py
├── mdp_estimator.py      # Core estimation functions
└── utils.py              # Helper functions (numerical Hessian, etc.)

test/mdp_estimator/
└── test_estimator.py     # Unit and integration tests
```

### Step 1: Create Estimator Module

File: `src/mdp_estimator/mdp_estimator.py`

```python
"""MDP Structural Parameter Estimator using NFXP."""

import numpy as np
import torch
from scipy import optimize
from dataclasses import dataclass
from typing import Tuple, Dict, Any, Optional

from mdp_solver import solve_value_function, compute_choice_probability
from mdp_simulator import PanelData


@dataclass
class EstimationResult:
    """Container for estimation results."""
    theta_hat: np.ndarray          # (beta, gamma, delta)
    std_errors: np.ndarray         # Standard errors
    cov_matrix: np.ndarray         # Covariance matrix
    log_likelihood: float          # Log-likelihood at optimum
    n_iterations: int              # Outer loop iterations
    converged: bool                # Whether optimizer converged
    optimization_result: Any       # Full scipy result object


def compute_log_likelihood(
    theta: np.ndarray,
    data: PanelData,
    solver_params: Dict[str, Any],
) -> float:
    """
    Compute log-likelihood for candidate parameters.
    
    Parameters
    ----------
    theta : np.ndarray
        Parameters (beta, gamma, delta)
    data : PanelData
        Panel data with states and actions
    solver_params : dict
        Solver hyperparameters (s_min, s_max, hidden_sizes, etc.)
    
    Returns
    -------
    float
        Log-likelihood value
    """
    beta, gamma, delta = theta
    
    # Validate bounds
    if beta <= 0 or gamma <= 0 or gamma >= 1 or delta <= 0 or delta >= 1:
        return -np.inf
    
    # Inner loop: solve for value functions
    v0_net, v1_net, losses, n_iter = solve_value_function(
        beta=beta,
        gamma=gamma,
        delta=delta,
        s_min=solver_params['s_min'],
        s_max=solver_params['s_max'],
        hidden_sizes=solver_params['hidden_sizes'],
        learning_rate=solver_params['learning_rate'],
        batch_size=solver_params['batch_size'],
        tolerance=solver_params['tolerance'],
        max_iterations=solver_params['max_iterations'],
        target_update_freq=solver_params.get('target_update_freq', 100),
    )
    
    # Compute choice probabilities
    states_flat = data.states.flatten()
    actions_flat = data.actions.flatten()
    s_tensor = torch.tensor(states_flat, dtype=torch.float32)
    
    with torch.no_grad():
        p1 = compute_choice_probability(v0_net, v1_net, s_tensor).numpy()
    
    # Clip for numerical stability
    eps = 1e-10
    p1 = np.clip(p1, eps, 1 - eps)
    
    # Log-likelihood
    log_lik = np.sum(
        actions_flat * np.log(p1) + 
        (1 - actions_flat) * np.log(1 - p1)
    )
    
    return log_lik


def estimate_mle(
    data: PanelData,
    theta_init: Tuple[float, float, float],
    solver_params: Dict[str, Any],
    maxiter: int = 200,
    verbose: bool = True,
) -> EstimationResult:
    """
    Estimate structural parameters via MLE using NFXP.
    
    Parameters
    ----------
    data : PanelData
        Panel data with states and actions
    theta_init : tuple
        Initial guess (beta, gamma, delta)
    solver_params : dict
        Solver hyperparameters
    maxiter : int
        Maximum outer loop iterations
    verbose : bool
        Print progress
    
    Returns
    -------
    EstimationResult
        Estimation results including estimates and standard errors
    """
    theta_init = np.array(theta_init)
    
    # Track evaluations
    eval_count = [0]
    
    def neg_log_lik(theta):
        eval_count[0] += 1
        ll = compute_log_likelihood(theta, data, solver_params)
        if verbose and eval_count[0] % 10 == 0:
            print(f"Eval {eval_count[0]}: theta={theta}, LL={ll:.2f}")
        return -ll
    
    # Run optimization
    result = optimize.minimize(
        neg_log_lik,
        theta_init,
        method='Nelder-Mead',
        options={
            'maxiter': maxiter,
            'xatol': 1e-4,
            'fatol': 1e-4,
            'disp': verbose,
            'adaptive': True,
        }
    )
    
    theta_hat = result.x
    log_lik_hat = -result.fun
    
    # Compute standard errors
    if verbose:
        print("Computing standard errors...")
    std_errors, cov_matrix = compute_standard_errors(
        theta_hat, data, solver_params
    )
    
    return EstimationResult(
        theta_hat=theta_hat,
        std_errors=std_errors,
        cov_matrix=cov_matrix,
        log_likelihood=log_lik_hat,
        n_iterations=result.nit,
        converged=result.success,
        optimization_result=result,
    )


def compute_standard_errors(
    theta_hat: np.ndarray,
    data: PanelData,
    solver_params: Dict[str, Any],
    eps: float = 1e-4,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Compute standard errors via numerical Hessian.
    
    Parameters
    ----------
    theta_hat : np.ndarray
        MLE estimates
    data : PanelData
        Panel data
    solver_params : dict
        Solver hyperparameters
    eps : float
        Step size for finite differences
    
    Returns
    -------
    std_errors : np.ndarray
        Standard errors for each parameter
    cov_matrix : np.ndarray
        Variance-covariance matrix
    """
    n_params = len(theta_hat)
    hessian = np.zeros((n_params, n_params))
    
    for i in range(n_params):
        for j in range(i, n_params):  # Exploit symmetry
            theta_pp = theta_hat.copy()
            theta_pp[i] += eps
            theta_pp[j] += eps
            
            theta_pm = theta_hat.copy()
            theta_pm[i] += eps
            theta_pm[j] -= eps
            
            theta_mp = theta_hat.copy()
            theta_mp[i] -= eps
            theta_mp[j] += eps
            
            theta_mm = theta_hat.copy()
            theta_mm[i] -= eps
            theta_mm[j] -= eps
            
            f_pp = compute_log_likelihood(theta_pp, data, solver_params)
            f_pm = compute_log_likelihood(theta_pm, data, solver_params)
            f_mp = compute_log_likelihood(theta_mp, data, solver_params)
            f_mm = compute_log_likelihood(theta_mm, data, solver_params)
            
            hessian[i, j] = (f_pp - f_pm - f_mp + f_mm) / (4 * eps * eps)
            hessian[j, i] = hessian[i, j]  # Symmetric
    
    # Information matrix = -Hessian
    info_matrix = -hessian
    
    # Covariance = inverse of information matrix
    try:
        cov_matrix = np.linalg.inv(info_matrix)
        std_errors = np.sqrt(np.diag(cov_matrix))
    except np.linalg.LinAlgError:
        # Singular matrix - return NaN
        cov_matrix = np.full((n_params, n_params), np.nan)
        std_errors = np.full(n_params, np.nan)
    
    return std_errors, cov_matrix
```

### Step 2: Add Tests

File: `test/mdp_estimator/test_estimator.py`

Key tests to implement:
1. **Unit test**: `compute_log_likelihood` returns correct sign and magnitude
2. **Unit test**: Standard errors are positive and reasonable
3. **Integration test**: Recover true parameters from simulated data
4. **Convergence test**: Optimizer converges for well-specified problems

### Step 3: Integration in Quarto Report

```python
# In estimate_mdp.qmd

# Load simulated data
from mdp_simulator import simulate_mdp_panel
from mdp_estimator import estimate_mle, EstimationResult

# Simulate data at true parameters
data = simulate_mdp_panel(...)

# Estimate with perturbed initial values
theta_init = (beta * 1.2, gamma * 0.8, delta * 0.95)  # Perturbed
result = estimate_mle(data, theta_init, solver_params)

# Display results
print(f"True:      β={beta:.3f}, γ={gamma:.3f}, δ={delta:.3f}")
print(f"Estimated: β={result.theta_hat[0]:.3f}, γ={result.theta_hat[1]:.3f}, δ={result.theta_hat[2]:.3f}")
print(f"Std Err:   ({result.std_errors[0]:.3f}, {result.std_errors[1]:.3f}, {result.std_errors[2]:.3f})")
```

## Implementation

### Setup

In [ ]:
#| label: setup
#| code-fold: true

import sys
import json
import numpy as np
import torch

# Add paths for local modules
sys.path.insert(0, '../../src')
sys.path.insert(0, '../config_mdp')

# Import configuration
from config import (
    beta as true_beta,
    gamma as true_gamma,
    delta as true_delta,
    s_min, s_max,
    hidden_sizes,
    learning_rate,
    batch_size,
    tolerance,
    max_iterations,
    target_update_freq,
)

# Import estimator
from mdp_estimator import grid_search_mle, EstimationResult
from mdp_simulator import PanelData

# Paths
SIMULATE_OUTPUT_PATH = '../../output/simulate_mdp'
SOLVER_OUTPUT_PATH = '../../output/solve_mdp'

print("Setup complete.")
print(f"True parameters: β={true_beta}, γ={true_gamma}, δ={true_delta}")

### Load Simulated Data

In [ ]:
#| label: load-data

# Load simulated panel data
states = np.load(f'{SIMULATE_OUTPUT_PATH}/states.npy')
actions = np.load(f'{SIMULATE_OUTPUT_PATH}/actions.npy')
rewards = np.load(f'{SIMULATE_OUTPUT_PATH}/rewards.npy')

# Load simulation config
with open(f'{SIMULATE_OUTPUT_PATH}/config.json', 'r') as f:
    sim_config = json.load(f)

n_agents = states.shape[0]
n_periods = states.shape[1]

# Create PanelData object
panel_data = PanelData(
    states=states,
    actions=actions,
    rewards=rewards,
    n_agents=n_agents,
    n_periods=n_periods,
)

print(f"Loaded panel data:")
print(f"  N agents:  {n_agents}")
print(f"  T periods: {n_periods}")
print(f"  Total observations: {n_agents * n_periods}")

### True Parameters (Ground Truth)

In [ ]:
#| label: true-params

print("=" * 50)
print("TRUE PARAMETERS (Data-Generating Process)")
print("=" * 50)
print(f"  β (reward coefficient):  {true_beta}")
print(f"  γ (state decay rate):    {true_gamma}")
print(f"  δ (discount factor):     {true_delta}")
print("=" * 50)

### Grid Search Estimation

In [ ]:
#| label: grid-search

# Solver parameters for inner loop
solver_params = {
    's_min': s_min,
    's_max': s_max,
    'hidden_sizes': hidden_sizes,
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'tolerance': tolerance,
    'max_iterations': max_iterations,
    'target_update_freq': target_update_freq,
}

# Grid search bounds (centered around true values with reasonable range)
bounds = {
    'beta': (0.5, 1.5),      # True: 1.0
    'gamma': (0.05, 0.20),   # True: 0.1
    'delta': (0.90, 0.99),   # True: 0.95
}

# Number of grid points per dimension
n_points = 5  # 5^3 = 125 evaluations

print("Starting grid search estimation...")
print(f"Grid: {n_points} points per dimension = {n_points**3} total evaluations")
print(f"Using warm-start from: {SOLVER_OUTPUT_PATH}")
print()

# Run grid search with warm-start
result = grid_search_mle(
    data=panel_data,
    bounds=bounds,
    n_points=n_points,
    solver_params=solver_params,
    verbose=True,
    compute_se=True,
    pretrained_path=SOLVER_OUTPUT_PATH,
)

print("\nGrid search completed!")

### Estimation Results

In [ ]:
#| label: results

import pandas as pd

# Extract results
beta_hat, gamma_hat, delta_hat = result.theta_hat
se_beta, se_gamma, se_delta = result.std_errors

# Create results table
results_df = pd.DataFrame({
    'Parameter': ['β (reward)', 'γ (decay)', 'δ (discount)'],
    'True': [true_beta, true_gamma, true_delta],
    'Estimated': [beta_hat, gamma_hat, delta_hat],
    'Std. Error': [se_beta, se_gamma, se_delta],
    'Bias': [beta_hat - true_beta, gamma_hat - true_gamma, delta_hat - true_delta],
    'Bias (%)': [
        100 * (beta_hat - true_beta) / true_beta,
        100 * (gamma_hat - true_gamma) / true_gamma,
        100 * (delta_hat - true_delta) / true_delta,
    ],
})

print("=" * 60)
print("ESTIMATION RESULTS")
print("=" * 60)
print(results_df.to_string(index=False))
print("=" * 60)
print(f"\nLog-likelihood at optimum: {result.log_likelihood:.2f}")
print(f"Number of evaluations: {result.n_iterations}")

### Likelihood at True Parameters

In [ ]:
#| label: likelihood-check

from mdp_estimator import compute_log_likelihood
import torch
import os

# Load pre-trained networks for warm-start
v0_init_state = torch.load(f'{SOLVER_OUTPUT_PATH}/v0_net.pt', weights_only=True)
v1_init_state = torch.load(f'{SOLVER_OUTPUT_PATH}/v1_net.pt', weights_only=True)

# Compute log-likelihood at TRUE parameters
theta_true = np.array([true_beta, true_gamma, true_delta])

print(f"Evaluating log-likelihood at true parameters: θ = {theta_true}")
ll_true = compute_log_likelihood(
    theta=theta_true,
    data=panel_data,
    solver_params=solver_params,
    v0_init_state=v0_init_state,
    v1_init_state=v1_init_state,
)

print(f"\nLog-likelihood comparison:")
print(f"  At TRUE parameters (β={true_beta}, γ={true_gamma}, δ={true_delta}):  LL = {ll_true:.2f}")
print(f"  At ESTIMATED parameters (β={beta_hat:.2f}, γ={gamma_hat:.2f}, δ={delta_hat:.2f}): LL = {result.log_likelihood:.2f}")
print(f"  Difference: {ll_true - result.log_likelihood:.2f}")

if ll_true > result.log_likelihood:
    print("\n⚠ TRUE parameters have HIGHER likelihood!")
    print("  → Grid search missed the true maximum (grid too coarse)")
elif abs(ll_true - result.log_likelihood) < 10:
    print("\n⚠ Likelihoods are SIMILAR (within 10 log-units)")
    print("  → Likelihood surface is FLAT - identification issues")
else:
    print("\n✓ Estimated parameters have higher likelihood")
    print("  → Model may be misspecified or data insufficient")

### Diagnostics

In [ ]:
#| label: diagnostics

print("=" * 60)
print("DIAGNOSTICS")
print("=" * 60)

# Check if estimates are within 2 standard errors of true values
within_2se = []
for i, (name, true_val, est_val, se) in enumerate([
    ('β', true_beta, beta_hat, se_beta),
    ('γ', true_gamma, gamma_hat, se_gamma),
    ('δ', true_delta, delta_hat, se_delta),
]):
    if np.isnan(se):
        status = "SE not available"
        within = None
    else:
        diff = abs(est_val - true_val)
        within = diff <= 2 * se
        status = "✓ Yes" if within else "✗ No"
    within_2se.append(within)
    print(f"  {name}: |{est_val:.4f} - {true_val:.4f}| = {abs(est_val - true_val):.4f} "
          f"{'<=' if within else '>'} 2×SE={2*se:.4f} → {status}")

print()

# Overall assessment
n_covered = sum(1 for w in within_2se if w is True)
n_total = sum(1 for w in within_2se if w is not None)

if n_total > 0:
    print(f"Coverage: {n_covered}/{n_total} parameters within 2 SE of true value")
    if n_covered == n_total:
        print("✓ All parameters recovered successfully!")
    else:
        print("⚠ Some parameters may have estimation issues.")
else:
    print("⚠ Standard errors not available for coverage assessment.")

print()
print("=" * 60)
print("COVARIANCE MATRIX")
print("=" * 60)
print(result.cov_matrix)